In [ ]:
from mne.stats import permutation_cluster_1samp_test
import numpy as np

import pickle
import sys

sys.path.append('..')

from _parameters import *
sys.path.remove('..')

In [ ]:
# Load TFR
cvsi_avg_f = open(dirs['cvsi'] + '/cvsi_enc1.pkl', 'rb')
cvsi_dat, cvsi_avg = pickle.load(cvsi_avg_f)

cvsi_avg_f.close()

# And time-courses
cvsi_tc_f = open(dirs['cvsi'] + '/cvsi_tc_enc1.pkl', 'rb')
cvsi_tc = pickle.load(cvsi_tc_f)

cvsi_tc_f.close()

In [ ]:
def stat_TFR(data):

    data = np.asarray(data)

    # Run cluster perm
    t, clust, p, h0 = permutation_cluster_1samp_test(data, n_permutations=10000, out_type= 'mask')

    # Create mask
    mask = np.asarray(clust)[p<.05]
    mask = np.kron(np.squeeze(mask), np.ones((10,10)))
    
    stat = {
        'mask': mask,
        'sig': p
    }

    return stat

In [ ]:
def stat_tc(data):

    time = cvsi_tc['time']

    t, clust, p, h0 = permutation_cluster_1samp_test(data, n_permutations=10000)

    # Get t-index of sign clusters
    sig_index = np.asarray(clust, dtype='object')[p<.05]

    # Create mask
    mask = []
    for i in range(len(sig_index)):
        mask = np.append(mask, (time[sig_index[i][0]]))

    return mask

In [ ]:
stats_TFR = {c:None for c in cvsi_dat.keys()}
stats_tc = {c:None for c in cvsi_dat.keys()}

# Run for each cond
for c in cvsi_dat.keys():

    tfr_dat = cvsi_dat[c]
    tc_dat = cvsi_tc[c]

    # Add stats to dict
    stats_TFR[c] = stat_TFR(tfr_dat)
    stats_tc[c] = stat_tc(tc_dat)

In [ ]:
# Save stats

stat_TFR_f = open(dirs['stat'] + '/stat_TFR_enc1.pkl', 'wb')
stat_tc_f = open(dirs['stat'] + '/stat_tc_enc1.pkl', 'wb')

pickle.dump(stats_TFR, stat_TFR_f)
stat_TFR_f.close()

pickle.dump(stats_tc, stat_tc_f)
stat_tc_f.close()